# Fully Open-Source Observability with Pydantic Logfire and Jaeger

Previously we used Logfire to send data to the Logfire backend.

The Logfire SDK is open source, and you can use it to export data to any OpenTelemetry-compatible backend.

The Logfire platform (the UI and backend) is closed source.

Key resources:

- Platform: https://pydantic.dev/logfire
- SDK: https://github.com/pydantic/logfire
- Alternative backends guide: https://logfire.pydantic.dev/docs/how-to-guides/alternative-backends/

This separation gives you flexibility - you can use the Logfire SDK with any observability backend that supports OpenTelemetry.

## Setting Up Jaeger

Let's add Jaeger as our open source observability backend. Here's the prompt I used:

```text
add jaeger to docker compose
```

Add this service to your Docker Compose file:

```yaml
# docker-compose.yml
  jaeger:
    image: jaegertracing/all-in-one:1.58
    container_name: monitoring-jaeger
    environment:
      - COLLECTOR_OTLP_ENABLED=true
    ports:
      - "16686:16686"   # Jaeger UI
      - "4317:4317"     # OTLP gRPC
      - "4318:4318"     # OTLP HTTP
      - "14268:14268"   # Jaeger Thrift HTTP (optional)
      - "14250:14250"   # Jaeger gRPC (optional)
```

Jaeger provides distributed tracing capabilities with a web UI for analyzing trace data.

Start it:

```bash
docker-compose up jaeger
```

You can open it at http://localhost:16686/.

## Configuring OpenTelemetry

Add these environment variables to your `.env` file:

```text
OTEL_EXPORTER_OTLP_TRACES_ENDPOINT='http://localhost:4318/v1/traces'
OTEL_SERVICE_NAME='evidently-docs'
```

You can read more about these variables here: https://opentelemetry.io/docs/specs/otel/configuration/sdk-environment-variables/

These variables configure the OpenTelemetry SDK to send traces to our local Jaeger instance.

## Updating Logfire Configuration

Replace your Logfire configuration:

In [ ]:
logfire.configure(send_to_logfire=False)

The `send_to_logfire=False` parameter prevents data from being sent to the Logfire platform. Instead, it uses the OpenTelemetry environment variables to send traces to Jaeger.

That's all! Now run your application.

## Alternative: Pure OpenTelemetry

Note that you can use OpenTelemetry directly without Logfire if needed for specific requirements. See the documentation: https://ai.pydantic.dev/logfire/#otel-without-logfire

However, the Logfire SDK provides excellent abstractions and PydanticAI integration while remaining vendor-neutral through OpenTelemetry standards.